# Profundización Región de Los Ríos — Oferta Formativa 2025

Este cuaderno desglosa en detalle la oferta académica de la Región de Los Ríos: niveles, modalidades, carreras y costos. Usamos Plotly para ofrecer gráficos interactivos y destacar oportunidades de planificación en esta región prioritaria.

In [1]:
from pathlib import Path
import unicodedata

import numpy as np
import pandas as pd
import plotly.express as px

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'data').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_PATH = PROJECT_ROOT / 'data/processed/oferta_formativa_clean.parquet'
if not DATA_PATH.exists():
    raise FileNotFoundError(f'No se encontró el archivo esperado: {DATA_PATH}')

df = pd.read_parquet(DATA_PATH)

def normalize(value: str) -> str:
    if not isinstance(value, str):
        return ''
    normalized = unicodedata.normalize('NFKD', value)
    return ''.join(ch for ch in normalized if not unicodedata.combining(ch)).lower()

mask = df['region_sede'].apply(normalize).str.contains('los rios', na=False)
los_rios = df.loc[mask].copy()
if los_rios.empty:
    raise ValueError('No hay registros para la Región de Los Ríos')

resumen_general = los_rios.agg({
    'codigo_carrera': 'nunique',
    'nombre_ies': 'nunique',
    'total_vacantes': 'sum',
    'arancel_anual': 'median',
    'matricula_anual': 'median'
}).rename({
    'codigo_carrera': 'programas_unicos',
    'nombre_ies': 'instituciones',
    'total_vacantes': 'vacantes_totales',
    'arancel_anual': 'arancel_median',
    'matricula_anual': 'matricula_median'
})
resumen_general

programas_unicos        236.0
instituciones             9.0
vacantes_totales       9877.0
arancel_median      2902000.0
matricula_median     250000.0
dtype: float64

## Cobertura de instituciones
Validamos todas las instituciones activas en la región y el total de programas que aportan.

In [7]:
ies_resumen = (
    los_rios.groupby('nombre_ies', as_index=False)
    .agg(programas=('codigo_carrera', 'nunique'), vacantes=('total_vacantes', 'sum'))
    .sort_values('programas', ascending=False)
)
print(f"Instituciones únicas en Los Ríos: {ies_resumen['nombre_ies'].nunique()}")
ies_resumen

Instituciones únicas en Los Ríos: 9


,nombre_ies,programas,vacantes
6,UNIVERSIDAD AUSTRAL DE CHILE,154,3418
7,UNIVERSIDAD SAN SEBASTIAN,39,2076
1,CFT INACAP,21,986
3,IP INACAP,15,504
0,CFT DE LA REGION DE LOS RIOS,11,1140
2,CFT SANTO TOMAS,11,525
4,IP IPG,9,625
8,UNIVERSIDAD SANTO TOMAS,8,320
5,IP SANTO TOMAS,6,283


## Vacantes totales por institución
Gráfico de barras que compara la capacidad de admisión agregada por universidad/IES en la región.

In [9]:
vacantes_ies = (
    los_rios.groupby('nombre_ies', as_index=False)
    .agg(vacantes_totales=('total_vacantes', 'sum'))
    .sort_values('vacantes_totales', ascending=True)
)
fig_ies = px.bar(
    vacantes_ies,
    x='vacantes_totales',
    y='nombre_ies',
    orientation='h',
    text='vacantes_totales',
    title='Vacantes totales por institución (Región de Los Ríos)',
    labels={'nombre_ies': 'Institución', 'vacantes_totales': 'Vacantes'}
)
fig_ies.update_layout(height=600)
fig_ies

In [11]:
# Export interactivo a HTML para compartir fuera del notebook
DOCS_DIR = PROJECT_ROOT / 'docs'
DOCS_DIR.mkdir(exist_ok=True)
fig_ies.write_html(
    DOCS_DIR / 'vacantes_por_institucion.html',
    include_plotlyjs='cdn',
    full_html=True
)
print('Archivo actualizado en docs/vacantes_por_institucion.html')

Archivo actualizado en docs/vacantes_por_institucion.html


## Vacantes por área del conocimiento
Permite identificar dónde se concentra la oferta regional y cuáles áreas podrían reforzarse.

In [2]:
vacantes_area = (
    los_rios.groupby('area_conocimiento', as_index=False)
    .agg(vacantes_totales=('total_vacantes', 'sum'))
    .sort_values('vacantes_totales', ascending=True)
)
fig_area = px.bar(
    vacantes_area,
    x='vacantes_totales',
    y='area_conocimiento',
    orientation='h',
    text='vacantes_totales',
    title='Vacantes totales por área del conocimiento',
    labels={'area_conocimiento': 'Área', 'vacantes_totales': 'Vacantes'}
)
fig_area.update_layout(height=700)
fig_area

## Distribución por nivel académico
Observamos cómo se reparte la oferta entre pregrado, posgrado y programas técnicos.

In [3]:
niveles = (
    los_rios.groupby('nivel_global', as_index=False)
    .agg(
        programas_unicos=('codigo_carrera', 'nunique'),
        vacantes_totales=('total_vacantes', 'sum')
    )
)
fig_nivel = px.bar(
    niveles,
    x='nivel_global',
    y='vacantes_totales',
    text='programas_unicos',
    title='Vacantes y programas por nivel global (texto = programas únicos)',
    labels={'nivel_global': 'Nivel', 'vacantes_totales': 'Vacantes'}
)
fig_nivel.update_traces(marker_color='#d62728')
fig_nivel

## Modalidades y jornadas
Gráfico apilado que combina modalidad (Presencial, Semipresencial, etc.) con jornada para diagnosticar la flexibilidad de la oferta.

In [4]:
modalidad = (
    los_rios.groupby(['modalidad', 'jornada'], as_index=False)
    .agg(vacantes_totales=('total_vacantes', 'sum'))
)
fig_modalidad = px.bar(
    modalidad,
    x='modalidad',
    y='vacantes_totales',
    color='jornada',
    title='Vacantes por modalidad y jornada',
    labels={'modalidad': 'Modalidad', 'vacantes_totales': 'Vacantes', 'jornada': 'Jornada'}
)
fig_modalidad.update_layout(barmode='stack')
fig_modalidad

## Top 15 carreras por vacantes
Muestra programas prioritarios según su capacidad de admisión.

## Carreras por institución
Visualización facetada con todas las universidades/IES de la región y sus carreras ordenadas por vacantes.

In [10]:
carreras_por_ies = (
    los_rios.groupby(['nombre_ies', 'nombre_carrera'], as_index=False)
    .agg(vacantes_totales=('total_vacantes', 'sum'))
)
fig_carreras_por_ies = px.bar(
    carreras_por_ies.sort_values(['nombre_ies', 'vacantes_totales'], ascending=[True, False]),
    x='vacantes_totales',
    y='nombre_carrera',
    color='nombre_ies',
    orientation='h',
    facet_row='nombre_ies',
    title='Vacantes por carrera dentro de cada institución (Región de Los Ríos)',
    labels={'nombre_carrera': 'Carrera', 'vacantes_totales': 'Vacantes', 'nombre_ies': 'IES'}
)
fig_carreras_por_ies.update_layout(
    height=3200,
    showlegend=False,
    margin={'l': 120, 'r': 30, 't': 80, 'b': 50}
)
fig_carreras_por_ies.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
fig_carreras_por_ies.update_yaxes(matches=None, automargin=True)
fig_carreras_por_ies

In [5]:
top_carreras = (
    los_rios.groupby(['nombre_carrera', 'nombre_ies'], as_index=False)
    .agg(vacantes_totales=('total_vacantes', 'sum'),
         arancel_promedio=('arancel_anual', 'mean'))
    .sort_values('vacantes_totales', ascending=False)
    .head(15)
)
fig_carreras = px.bar(
    top_carreras,
    x='vacantes_totales',
    y='nombre_carrera',
    color='nombre_ies',
    text='vacantes_totales',
    orientation='h',
    title='Top 15 carreras por vacantes (Región de Los Ríos)',
    labels={'nombre_carrera': 'Carrera', 'vacantes_totales': 'Vacantes', 'nombre_ies': 'IES'}
)
fig_carreras.update_layout(height=750, yaxis={'categoryorder': 'total ascending'})
fig_carreras

## Costos medianos por institución
Comparación en línea de la matrícula y el arancel medianos por institución para detectar brechas de precio dentro de la región.

In [8]:
costos_linea = (
    los_rios.groupby('nombre_ies', as_index=False)
    .agg(
        arancel_median=('arancel_anual', 'median'),
        matricula_median=('matricula_anual', 'median')
    )
    .sort_values('arancel_median', ascending=False)
)
series_costos = costos_linea.melt(
    id_vars='nombre_ies',
    value_vars=['arancel_median', 'matricula_median'],
    var_name='tipo',
    value_name='monto'
)
fig_costos = px.line(
    series_costos,
    x='nombre_ies',
    y='monto',
    color='tipo',
    markers=True,
    title='Comparación de matrícula y arancel medianos por institución',
    labels={'nombre_ies': 'Institución', 'monto': 'CLP', 'tipo': 'Indicador'}
)
fig_costos.update_layout(xaxis_tickangle=-35)
fig_costos